In [1]:
import os

In [2]:
from git import Repo

In [5]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

### Clone a GITHUB Repo ###

In [7]:
%pwd

'/Users/deltae/Source-Code-Analysis-RAG/research'

In [8]:
!mkdir test_repo 

In [10]:
repo_path = "/Users/deltae/Source-Code-Analysis-RAG/research/test_repo"

In [11]:
Repo.clone_from("https://github.com/architthakur2000/Convulational-Neural-Networks.git", to_path=repo_path)

<git.repo.base.Repo '/Users/deltae/Source-Code-Analysis-RAG/research/test_repo/.git'>

In [14]:
loader = GenericLoader.from_filesystem(repo_path, glob = "**/*", suffixes = [".py"], parser = LanguageParser(language=Language.PYTHON, parser_threshold=500))

In [15]:
documents = loader.load()

In [16]:
documents

[Document(page_content='# python imports\nimport os\nimport time\nimport argparse\nfrom tqdm import tqdm\n\n# torch imports\nimport torch\nimport torch.nn as nn\nimport torch.optim as optim\n\n# helper functions for computer vision\nimport torchvision\nimport torchvision.transforms as transforms\n\nfrom dataloader import MiniPlaces\nfrom student_code import LeNet, test_model\n\n\n# main function for training and testing\ndef main(args):\n    # set up random seed\n    torch.manual_seed(0)\n\n    ###################################\n    # setup model                     #\n    ###################################\n    model = LeNet()\n    # set up transforms to transform the PIL Image to tensors\n    test_transform = transforms.Compose([\n        transforms.ToTensor(),\n        transforms.Normalize(mean=[0.485, 0.456, 0.406],\n                             std=[0.229, 0.224, 0.225])\n    ])\n\n    ################################\n    # setup dataset and dataloader #\n    #################

### Chunking and Splitting ###

In [20]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, chunk_size = 2000, chunk_overlap = 200)

In [21]:
texts = documents_splitter.split_documents(documents)

In [22]:
len(texts)

16

### Embedding Model ###

In [23]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [25]:
embeddings = OpenAIEmbeddings(disallowed_special=())

In [26]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory="./sca_db")

In [27]:
vectordb.persist()

In [28]:
llm = ChatOpenAI()

In [29]:
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

In [30]:
chain = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)

### Tests ###

In [31]:
question = "Explain the train_model function in student_code.py"

In [32]:
result = chain(question)

Number of requested results 20 is greater than number of elements in index 16, updating n_results = 16


In [33]:
print(result["answer"])

The `train_model` function in the provided code snippet is used for training a PyTorch model. It takes the model, training data loader, optimizer, loss function (criterion), and the current epoch number as input. Inside the function, it performs the standard training loop steps which include forward pass, backward pass, loss computation, and optimizer step to update the model parameters.

Here is a brief overview of the key steps within the `train_model` function:
1. Sets the model to training mode.
2. Iterates through the training data loader to process each batch.
3. Computes the loss based on the model's output and the target labels.
4. Backpropagates the loss to compute gradients and updates the model's parameters using the optimizer.
5. Tracks and accumulates the training loss for each epoch.
6. Prints the average training loss for the current epoch.

Overall, the `train_model` function is responsible for training the model over multiple epochs by iterating through the training da